In [ ]:
# ==========================================
# 0. IMPORTS
# ==========================================

#probando si esto se commitea Segunda prueba desde otro pc
#Nota de intiti: si van a trabajar desde un entorno local (Visual), 
# asegúrense de tener instaladas las librerías necesarias.
#tutorial: ctrl + ñ para abrir el terminal y luego pegar los siguientes comandos:
#comando para instalar torch: pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu121 -> En caso que quieran usar GPU.
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import TensorDataset, DataLoader
import numpy as np
import os
import glob
import math
from tqdm import tqdm
import os
import requests 
import gc # Garbage Collector para gestión de memoria

In [2]:
# ==========================================
# 1. CONFIGURACIÓN, GPU Y DESCARGA DE DATOS
# ==========================================


# --- A. CONFIGURACIÓN DEL HARDWARE (DEVICE) ---
# Esto es vital para que el Bloque de entrenamiento sepa qué usar
if torch.cuda.is_available():
    DEVICE = torch.device("cuda")
    print(f"✅ GPU DETECTADA: {torch.cuda.get_device_name(0)}")
    print(f"   (Memoria disponible: {torch.cuda.get_device_properties(0).total_memory / 1e9:.2f} GB)")
else:
    DEVICE = torch.device("cpu")
    print("⚠️ GPU NO DETECTADA: Entrenando en CPU (será lento).")

# --- B. CONFIGURACIÓN DEL REPOSITORIO ---
REPO_USER = "felipe-astudillo-s"
REPO_NAME = "TransformerTSP"
BRANCH = "main" # ⚠️ IMPORTANTE: Si tus datos no están en 'main', cambia esto por el nombre de tu rama o commit.

REPO_FOLDERS = {
    "EASY":   "Data/Easy",
    "MEDIUM": "Data/Medium",
    "HARD":   "Data/Hard"
}

BASE_LOCAL_DIR = os.path.join(os.getcwd(), "data_repo")

def download_folder_from_github(user, repo, repo_folder_path, local_output_dir, branch="main"):
    """Descarga todos los .npz de una carpeta de GitHub usando la API."""
    api_url = f"https://api.github.com/repos/{user}/{repo}/contents/{repo_folder_path}?ref={branch}"
    
    print(f"🔍 Consultando API para: {repo_folder_path}...")
    try:
        response = requests.get(api_url)
        if response.status_code == 404:
            print(f"❌ Error 404: No existe la carpeta '{repo_folder_path}' en la rama '{branch}'.")
            return local_output_dir
        if response.status_code != 200:
            print(f"❌ Error API ({response.status_code}): {response.text}")
            return local_output_dir

        files_list = response.json()
        
        if not os.path.exists(local_output_dir):
            os.makedirs(local_output_dir)

        if isinstance(files_list, dict) and 'message' in files_list:
            print("❌ Error: La ruta parece no ser una carpeta válida.")
            return local_output_dir

        count = 0
        for item in files_list:
            if item['type'] == 'file' and item['name'].endswith('.npz'):
                local_path = os.path.join(local_output_dir, item['name'])
                if not os.path.exists(local_path):
                    try:
                        r = requests.get(item['download_url'])
                        with open(local_path, 'wb') as f:
                            f.write(r.content)
                        count += 1
                    except Exception as e:
                        print(f"  ❌ Falló {item['name']}: {e}")
                else:
                    count += 1 # Ya existía
        
        print(f"✅ Fase {repo_folder_path}: {count} archivos listos en {local_output_dir}")
        return local_output_dir

    except Exception as e:
        print(f"❌ Error de conexión: {e}")
        return local_output_dir

# --- C. EJECUCIÓN DE DESCARGA ---
PATHS = {}
print(f"\n⚙️ Sincronizando con GitHub ({REPO_USER}/{REPO_NAME})...")

for phase_name, repo_path in REPO_FOLDERS.items():
    local_target = os.path.join(BASE_LOCAL_DIR, phase_name)
    final_path = download_folder_from_github(REPO_USER, REPO_NAME, repo_path, local_target, BRANCH)
    PATHS[phase_name] = final_path

# --- D. CURRICULUM ---
CURRICULUM = [
    {"phase": "EASY",   "epochs": 20, "lr": 1e-3, "bs": 128},
    {"phase": "MEDIUM", "epochs": 15, "lr": 1e-4, "bs": 64},
    {"phase": "HARD",   "epochs": 30, "lr": 1e-4, "bs": 32}
]

print(f"\n📂 Rutas configuradas correctamente.")
print(f"🚀 Listo para ejecutar el Bloque de Entrenamiento.")

✅ GPU DETECTADA: NVIDIA GeForce RTX 5060
   (Memoria disponible: 8.55 GB)

⚙️ Sincronizando con GitHub (felipe-astudillo-s/TransformerTSP)...
🔍 Consultando API para: Data/Easy...
❌ Error 404: No existe la carpeta 'Data/Easy' en la rama 'main'.
🔍 Consultando API para: Data/Medium...
✅ Fase Data/Medium: 20 archivos listos en d:\Proyectos\Transformer_TSP\TransformerTSP\data_repo\MEDIUM
🔍 Consultando API para: Data/Hard...
❌ Error 404: No existe la carpeta 'Data/Hard' en la rama 'main'.

📂 Rutas configuradas correctamente.
🚀 Listo para ejecutar el Bloque de Entrenamiento.


In [3]:

# ==========================================
# 2. ARQUITECTURA DEL MODELO (POINTER NETWORK)
# ==========================================

# ENCODER (Sin Positional Encoding y con return memory)
class Encoder(nn.Module):
    def __init__(self, input_dim, d_model=128, nhead=8, num_layers=3, dim_feedforward=512, dropout=0.1):
        super().__init__()
        self.input_proj = nn.Linear(input_dim, d_model)

        encoder_layer = nn.TransformerEncoderLayer(
            d_model=d_model,
            nhead=nhead,
            dim_feedforward=dim_feedforward,
            dropout=dropout,
            batch_first=True
        )
        self.encoder = nn.TransformerEncoder(encoder_layer, num_layers=num_layers)
        self.d_model = d_model

    def forward(self, x, src_key_padding_mask=None):
        # x: [batch, seq_len, input_dim]
        h = self.input_proj(x)  # [B, S, d_model]

        memory = self.encoder(h, src_key_padding_mask=src_key_padding_mask)

        return memory


# --- 2. DECODER
class PointerDecoder(nn.Module):
    def __init__(self, d_model=128, nhead=8, num_layers=2, dropout=0.1, max_seq_len=128):
        super().__init__()
        self.start_token = nn.Parameter(torch.randn(1, 1, d_model))
        self.step_emb = nn.Embedding(max_seq_len, d_model)

        decoder_layer = nn.TransformerDecoderLayer(
            d_model=d_model,
            nhead=nhead,
            dim_feedforward=d_model*4,
            dropout=dropout,
            batch_first=True
        )
        self.decoder = nn.TransformerDecoder(decoder_layer, num_layers=num_layers)
        self.query_proj = nn.Linear(d_model, d_model)

    def forward(self, memory, tgt_indices=None, mask_visited=None, teacher_forcing=True):
        B, S, d = memory.size()
        device = memory.device
        max_T = tgt_indices.size(1) if (tgt_indices is not None and teacher_forcing) else S

        start = self.start_token.expand(B, -1, -1)
        logits_steps = []
        decoder_inputs = start
        current_mask = torch.zeros(B, S, dtype=torch.bool).to(device)

        for t in range(max_T):
            step_emb = self.step_emb(torch.tensor([t], device=device)).unsqueeze(0).expand(B, -1, -1)
            dec_in = decoder_inputs + step_emb
            dec_out = self.decoder(dec_in, memory, memory_key_padding_mask=None)

            q_t = dec_out[:, -1, :]
            q = self.query_proj(q_t).unsqueeze(1)

            scores = torch.matmul(q, memory.transpose(1,2)) / math.sqrt(d)
            scores = scores.squeeze(1)

            if not teacher_forcing:
                scores = scores.masked_fill(current_mask, float('-inf'))

            logits_steps.append(scores)

            if teacher_forcing and tgt_indices is not None:
                idx_t = tgt_indices[:, t]
            else:
                probs = F.softmax(scores, dim=-1)
                idx_t = probs.argmax(dim=-1)
                new_visit = F.one_hot(idx_t, num_classes=S).bool()
                current_mask = current_mask | new_visit

            next_emb = torch.gather(memory, 1, idx_t.view(B,1,1).expand(-1,1,d)).squeeze(1).unsqueeze(1)
            decoder_inputs = torch.cat([decoder_inputs, next_emb], dim=1)

        return torch.stack(logits_steps, dim=1)


#  MODELO PRINCIPAL
class EncoderPointerModel(nn.Module):
    def __init__(self, input_dim=2, d_model=128, enc_layers=3, dec_layers=2, nhead=8, max_seq_len=128):
        super().__init__()
        # CORRECCIÓN: Encoder ya no recibe max_seq_len
        self.encoder = Encoder(
            input_dim=input_dim,
            d_model=d_model,
            nhead=nhead,
            num_layers=enc_layers
        )
        self.decoder = PointerDecoder(
            d_model=d_model,
            nhead=nhead,
            num_layers=dec_layers,
            max_seq_len=max_seq_len
        )
        self.d_model = d_model

    def forward(self, x, tgt_indices=None, mask_padding=None,
                mask_visited=None, teacher_forcing=True,
                return_probabilities=False):

        memory = self.encoder(x, src_key_padding_mask=mask_padding)

        logits = self.decoder(
            memory,
            tgt_indices=tgt_indices,
            mask_visited=mask_visited,
            teacher_forcing=teacher_forcing
        )

        if return_probabilities:
            return torch.softmax(logits, dim=-1)
        return logits

In [4]:

# ==========================================
# 3. UTILIDADES DE EVALUACIÓN
# ==========================================
def calculate_gap(model, loader, device):
    """Calcula el Optimality GAP (%) usando Greedy Decoding en un batch."""
    model.eval()
    try:
        # Tomamos solo el primer batch para no demorar el entrenamiento
        batch_x, batch_y = next(iter(loader))
    except StopIteration:
        return 0.0 # Loader vacío

    batch_x, batch_y = batch_x.to(device), batch_y.to(device)
    batch_size, n_nodes, _ = batch_x.size()

    with torch.no_grad():
        # Inferencia Greedy (Teacher Forcing = False)
        # El modelo genera la secuencia de índices automáticamente
        logits = model(batch_x, teacher_forcing=False)
        # logits: [Batch, N, N_nodes]

        pred_indices = logits.argmax(dim=2) # [Batch, N]

        # Stackear para formar tour
        pred_tour = pred_indices

    # --- Cálculo de Distancias ---
    def get_dist(pts, idx):
        # pts: [B, N, 2], idx: [B, N]
        gathered = torch.gather(pts, 1, idx.unsqueeze(-1).expand(-1, -1, 2))
        next_pts = torch.roll(gathered, -1, dims=1)
        return torch.norm(gathered - next_pts, dim=2).sum(dim=1)

    cost_model = get_dist(batch_x, pred_tour)
    cost_oracle = get_dist(batch_x, batch_y)

    gap = ((cost_model - cost_oracle) / cost_oracle).mean().item() * 100
    return gap

In [5]:

# ==========================================
# 4. BUCLE DE ENTRENAMIENTO (LAZY LOADING)
# ==========================================

# Instanciar Modelo con la nueva clase
model = EncoderPointerModel(input_dim=2, d_model=128, nhead=8, enc_layers=3, dec_layers=2, max_seq_len=150).to(DEVICE) # Ajusta max_seq_len según tus datos más grandes
criterion = nn.CrossEntropyLoss()

print("\n🚀 INICIANDO ENTRENAMIENTO SOTA (Multi-Archivo)")

for stage in CURRICULUM:
    phase = stage['phase']
    folder_path = PATHS[phase]

    print(f"\n{'='*60}")
    print(f"🎓 FASE ACTUAL: {phase} | Epochs: {stage['epochs']}")
    print(f"{'='*60}")

    # Buscar archivos .npz y .tpz
    all_files = glob.glob(os.path.join(folder_path, "*.npz")) + \
                glob.glob(os.path.join(folder_path, "*.tpz"))

    if not all_files:
        print(f"⚠️ ALERTA: No encontré datos en {folder_path}. Saltando fase.")
        continue

    print(f"📂 Archivos detectados: {len(all_files)}")

    optimizer = optim.Adam(model.parameters(), lr=stage['lr'])
    scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, 'min', patience=3, factor=0.5)

    for epoch in range(stage['epochs']):
        model.train()
        epoch_loss_accum = 0
        total_batches = 0
        current_gap = 0

        # --- BUCLE SOBRE ARCHIVOS (Lazy Loading) ---
        for file_idx, file_path in enumerate(all_files):
            try:
                # 1. Cargar Archivo a RAM
                data = np.load(file_path)
                points = torch.FloatTensor(data['points'])
                solutions = torch.LongTensor(data['solutions'])

                # Normalización defensiva
                if points.max() > 1.0: points /= points.max()

                dataset = TensorDataset(points, solutions)
                loader = DataLoader(dataset, batch_size=stage['bs'], shuffle=True)

                # 2. Entrenar sobre este archivo
                pbar = tqdm(loader, desc=f"Ep {epoch+1} | {os.path.basename(file_path)}", leave=False)

                for batch_x, batch_y in pbar:
                    batch_x, batch_y = batch_x.to(DEVICE), batch_y.to(DEVICE)
                    optimizer.zero_grad()

                    # Teacher Forcing: Pasamos la solución completa (batch_y) como target
                    logits = model(batch_x, tgt_indices=batch_y, teacher_forcing=True)
                    # logits: [Batch, Seq, N_ciudades]

                    # Aplanar para Loss
                    # logits.reshape(-1, logits.size(-1)) -> [Batch*Seq, N_ciudades]
                    # batch_y.reshape(-1) -> [Batch*Seq]
                    loss = criterion(logits.reshape(-1, logits.size(-1)), batch_y.reshape(-1))

                    loss.backward()
                    torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)
                    optimizer.step()

                    epoch_loss_accum += loss.item()
                    pbar.set_postfix({'loss': loss.item()})

                total_batches += len(loader)

                # Calcular GAP solo en el último archivo de la época para ahorrar tiempo
                if file_idx == len(all_files) - 1:
                    current_gap = calculate_gap(model, loader, DEVICE)

                # 3. LIMPIEZA DE MEMORIA
                del data, points, solutions, dataset, loader
                gc.collect()
                torch.cuda.empty_cache()

            except Exception as e:
                print(f"❌ Error leyendo archivo {file_path}: {e}")
                continue

        # --- REPORTE DE ÉPOCA ---
        avg_loss = epoch_loss_accum / total_batches if total_batches > 0 else 0
        print(f"    📉 Epoca {epoch+1} Terminada | Loss: {avg_loss:.4f} | 📊 GAP: {current_gap:.2f}%")

        # Scheduler Step
        scheduler.step(avg_loss)

        # Guardar Checkpoint
        save_file = os.path.join(folder_path, f"checkpoint_{phase}_best.pth")
        torch.save(model.state_dict(), save_file)

print("\n🏆 ENTRENAMIENTO COMPLETADO EXITOSAMENTE.")


🚀 INICIANDO ENTRENAMIENTO SOTA (Multi-Archivo)

🎓 FASE ACTUAL: EASY | Epochs: 20
⚠️ ALERTA: No encontré datos en d:\Proyectos\Transformer_TSP\TransformerTSP\data_repo\EASY. Saltando fase.

🎓 FASE ACTUAL: MEDIUM | Epochs: 15
📂 Archivos detectados: 20


    📉 Epoca 1 Terminada | Loss: 2.3547 | 📊 GAP: 27.01%


    📉 Epoca 2 Terminada | Loss: 1.9000 | 📊 GAP: 28.11%


KeyboardInterrupt: 